In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from math import log

In [2]:
class SONAR(object):
    '''
    sonar数据集
    '''
    def __init__(self,root,path):
        '''
        方法说明:
            初始化类
        参数说明:
            root: 文件夹根目录
            path: sonar数据集文件名 'sonar.csv'
        '''
        self.root = root
        self.path = path
        self.data = self._get_data()
        self.label = self._get_label()

    def _get_data(self):
        #打开数据集
        with open(os.path.join(self.root,self.path),'r') as f:
            data = f.readlines()
        #去除掉sonar逗号
        for i in range(len(data)):
            data[i] = data[i].strip().split(',')[:-1]
        #转化为numpy数组格式
        data = np.array(data, dtype = float)
        
        return data

    def _get_label(self):
        #打开数据集
        with open(os.path.join(self.root,self.path),'r') as f:
            data = f.readlines()
        #'R'为0, 'M'为1
        label = [0] * len(data)
        for i in range(len(data)):
            if data[i][-2] == 'R':
                label[i] = 0
            else:
                label[i] = 1
        #转化为numpy数组格式
        label = np.array(label, dtype = int)
        
        return label

In [3]:
def ent(D):
    # 计算数据集D的信息熵
    n = D.shape[0]
    n0 = (D[:, -1] == 0).sum()
    p0 = n0 / n
    p1 = 1 - p0
    if p0 == 0 or p1 == 0:
        return 1
    else:
        return -(p0 * log(p0, 2) + p1 * log(p1, 2))

In [4]:
def T(D, a):
    '''
        D: 数据集
        a: a属性列的序数
        return: 对连续属性a的候选划分点集合t
    '''
    a_col = np.unique(D[:, a])
    n = a_col.shape[0]
    t = np.empty(n - 1)
    for i in range(1, n):
        t[i - 1] = (a_col[i - 1] + a_col[i]) / 2
    return t

In [5]:
def devide(D, A, T):
    '''
        D: 数据集
        A: 属性集
        T: 对于所有属性的各自的划分点集合
        return: 对属性集A的划分结果A_T
    '''
    n = A.shape[0]
    A_T = np.empty(D.shape)
    for i in range(n):
        A_T[D[:, A[i]] > T[i], A[i]] = 1
        A_T[D[:, A[i]] <= T[i], A[i]] = 0
    return A_T

In [6]:
def gain_t(D, a, T):
    '''
        D：数据集
        a：a属性列的序数
        T: 划分点集合
        return: 划分点集合T中最大的增益, 使增益最大的划分点t
    '''
    global choosen_t # 若不将其声明为全局变量，在预剪枝算法中会报错，原因未知
    n = T.shape[0]
    maxx = -999

    for i in range(n):
        t = T[i]
        posi =  D[D[:, a] > t, :]
        nega = D[D[:, a] <= t, :]
        gain = ent(D) - posi.shape[0] * ent(posi) / D.shape[0] - nega.shape[0] * ent(nega) / D.shape[0]
        if gain > maxx:
            maxx = gain
            choosen_t = t
    return maxx, choosen_t

In [7]:
class Tree_Node(object):
    def __init__(self, D):
        self.label = None
        self.a = None
        self.t = None
        self.D = D
        self.children = [0] * 2
        self.is_leaf = True

In [8]:
class Decision_Tree(object):
    '''
    决策树模型(ID3)
    '''
    def __init__(self):
        '''
        方法说明:
            初始化类
        '''
        self.root = None #决策树根
        self.T = None # 保存对于所有属性的各自的划分点集合

    def generate(self, D, A):
        '''
            进行决策树结点的生成
            D:训练集
            A:属性集
        '''
        node = Tree_Node(D)
        n = D.shape[0]
        n0 = (D[:, -1] == 0).sum()
        n1 = (D[:, -1] == 1).sum()
        # 若D中样本全属于同一类别，则将当前结点设为叶子结点：
        if n == n0:
            node.label = 0
            return node
        if n == n1:
            node.label = 1
            return node    
        # 若D中样本属性值全部相同并且都已做出划分，则将当前结点设为叶子结点：           
        if np.unique(devide(D, A, self.T), axis = 0).shape[0] == 1 and (self.T == 0).sum() == 0:
            if n1 >= n0:
                node.label = 1
            else:
                node.label = 0
            return node
        # 不满足以上两种情况，当前结点产生子树：
        # 选出使信息增益最大的属性及其划分值
        max = -1
        for i in range(A.shape[0]):
            a = A[i]
            gain, t = gain_t(D, a, T(D, a))
            if gain > max:
                max = gain
                best_a = a
                best_t = t       
        # 给当前节点赋值
        node.a = best_a
        node.t = best_t
        node.is_leaf = False
        self.T[node.a] = node.t
        D0 = D[D[:, node.a] <= node.t, :]
        D1 = D[D[:, node.a] >= node.t, :]
        # 给子树赋值
        child0 = Tree_Node(D0)
        child1 = Tree_Node(D1)
        # 若子树样本全部属于同一类别，则将子树设为叶子结点，否则在子树上继续执行生成算法：
        if D0.shape[0] == 0:
            n = D0.shape[0]
            n0 = (D0[:, -1] == 0).sum()
            n1 = (D0[:, -1] == 1).sum()
            if n1 >= n0:
                child0.label = 1
            else:
                child0.label = 0
        
            node.children[0] = child0

        else:
            node.children[0] = self.generate(D0, A)

        if D1.shape[0] == 0:
            n = D1.shape[0]
            n0 = (D1[:, -1] == 0).sum()
            n1 = (D1[:, -1] == 1).sum()
            if n1 >= n0:
                child1.label = 1
            else:
                child1.label = 0
            
            node.children[1] = child1

        else:
            node.children[1] = self.generate(D1, A)
        
        return node

    def train(self, D, A):
        '''
            以D和A训练决策树
            D:训练集
            A:属性集
        '''
        self.T = np.zeros(A.shape[0]) #将属性的划分值集合初始化为0
        self.root = self.generate(D, A) #从根结点开始生成决策树

    def predict(self, D, A):
        '''
            通过D和A，在训练好的决策树上进行预测
            D:训练集
            A:属性集
        '''
        n = D.shape[0]
        y = np.empty(n)
        for i in range(n):
            node = self.root
            while node.is_leaf == False:
                if(D[i, node.a] <= node.t):
                    node = node.children[0]
                else:
                    node = node.children[1]
            
            y[i] = node.label

        return y

In [9]:
# 读取数据
dataset = SONAR('', 'sonar.csv')
X = dataset.data
y = dataset.label

In [10]:
# 合并为数据集
D = np.hstack([X, y.reshape(-1, 1)])
A = np.arange(1, D.shape[1] - 1)

In [11]:
# 以7:3的比例划分训练集与测试集
np.random.seed(0)   # seed = 0
np.random.shuffle(D)
d = int(D.shape[0] * 0.7) 
D_train = D[: d]
D_test = D[d: ]

In [12]:
dt = Decision_Tree()

In [13]:
dt.train(D_train, A)

In [14]:
y_predict = dt.predict(D_test, A)

In [15]:
precision = np.sum(y_predict == D_test[:, -1]) / D_test.shape[0]

In [16]:
precision

0.6984126984126984

In [17]:
# 在训练集的基础上再以7:3的比例划分训练集中的训练集与测试集
np.random.shuffle(D_train)
d = int(D_train.shape[0] * 0.7) 
D_train_train = D_train[: d]
D_train_test = D_train[d: ]

In [18]:
class Decision_Tree_prepruning(object):
    '''
    预剪枝决策树模型(ID3)
    '''
    def __init__(self):
        '''
        方法说明:
            初始化类
        '''
        self.root = None #决策树根
        self.T = None # 保存对于所有属性的各自的划分点集合

    def generate(self, D, A):
        '''
            进行决策树结点的生成
            D:训练集
            A:属性集
        '''

        node = Tree_Node(D)
        n = D.shape[0]
        n0 = (D[:, -1] == 0).sum()
        n1 = (D[:, -1] == 1).sum()
        # 若D中样本全属于同一类别，则将当前结点设为叶子结点：
        if n == n0:
            node.label = 0
            return node
        if n == n1:
            node.label = 1
            return node    
        # 若D中样本属性值全部相同并且都已做出划分，则将当前结点设为叶子结点：           
        if np.unique(devide(D, A, self.T), axis = 0).shape[0] == 1 and (self.T == 0).sum() == 0:
            if n1 >= n0:
                node.label = 1
            else:
                node.label = 0
            return node
        # 不满足以上两种情况，进行预剪枝，判断是否应该使得当前结点产生子树：
        # 1:计算不产生子树情况下，测试集准确度：
        n = D.shape[0]
        n0 = (D[:, -1] == 0).sum()
        n1 = (D[:, -1] == 1).sum()
        if n1 >= n0:
            node.label = 1
        else:
            node.label = 0
        if self.root == None:
            self.root = node
        y_predict = dt.predict(D_train_test, A)
        precision_nodevide = np.sum(y_predict == D_train_test[:, -1]) / D_train_test.shape[0]

        # 选出使信息增益最大的属性及其划分值
        m = -1
        for i in range(A.shape[0]):
            a = A[i]
            gain, t = gain_t(D, a, T(D, a))
            if gain > m:
                m = gain
                best_a = a
                best_t = t       
        # 给当前节点赋值
        node.a = best_a
        node.t = best_t
        
        self.T[node.a] = node.t
        D0 = D[D[:, node.a] <= node.t, :]
        D1 = D[D[:, node.a] >= node.t, :]
        # 给子树赋值
        child0 = Tree_Node(D0)
        child1 = Tree_Node(D1)
        # 若子树样本全部属于同一类别，则将子树设为叶子结点，否则在子树上继续执行生成算法：
        if D0.shape[0] == 0:
            n = D0.shape[0]
            n0 = (D0[:, -1] == 0).sum()
            n1 = (D0[:, -1] == 1).sum()
            if n1 >= n0:
                child0.label = 1
            else:
                child0.label = 0
        
            node.children[0] = child0

        else:
            node.children[0] = self.generate(D0, A)

        if D1.shape[0] == 0:
            n = D1.shape[0]
            n0 = (D1[:, -1] == 0).sum()
            n1 = (D1[:, -1] == 1).sum()
            if n1 >= n0:
                child1.label = 1
            else:
                child1.label = 0
            
            node.children[1] = child1

        else:
            node.children[1] = self.generate(D1, A)

        if self.root == None:
            self.root = node
        node.is_leaf = False
        y_predict = dt.predict(D_train_test, A)
        precision_devide = np.sum(y_predict == D_train_test[:, -1]) / D_train_test.shape[0]

        if(precision_nodevide > precision_devide):
            node.is_leaf = True
            n = D.shape[0]
            n0 = (D[:, -1] == 0).sum()
            n1 = (D[:, -1] == 1).sum()
            if n1 >= n0:
                node.label = 1
            else:
                node.label = 0
            return node
        else:
            node.is_leaf = False
        
        return node

    def train(self, D, A):
        '''
            以D和A训练决策树
            D:训练集
            A:属性集
        '''
        self.T = np.zeros(A.shape[0]) #将属性的划分值集合初始化为0
        self.root = self.generate(D, A) #从根结点开始生成决策树

    def predict(self, D, A):
        '''
            通过D和A，在训练好的决策树上进行预测
            D:训练集
            A:属性集
        '''
        n = D.shape[0]
        y = np.empty(n)
        for i in range(n):
            node = self.root
            while node.is_leaf == False:
                if(D[i, node.a] <= node.t):
                    node = node.children[0]
                else:
                    node = node.children[1]
            
            y[i] = node.label

        return y

In [19]:
dt_p = Decision_Tree_prepruning()

In [20]:
dt_p.train(D_train_train, A)

In [21]:
y_predict = dt_p.predict(D_test, A)

In [22]:
precision = np.sum(y_predict == D_test[:, -1]) / D_test.shape[0]

In [23]:
precision

0.7142857142857143

In [24]:
# 在训练集的基础上再以6:4的比例划分训练集中的训练集与测试集
np.random.shuffle(D_train)
d = int(D_train.shape[0] * 0.6) 
D_train_train = D_train[: d]
D_train_test = D_train[d: ]

In [25]:
dt_p = Decision_Tree_prepruning()
dt_p.train(D_train_train, A)
y_predict = dt_p.predict(D_test, A)
precision = np.sum(y_predict == D_test[:, -1]) / D_test.shape[0]
precision

0.6666666666666666

In [26]:
# 在训练集的基础上再以8:2的比例划分训练集中的训练集与测试集
np.random.shuffle(D_train)
d = int(D_train.shape[0] * 0.8) 
D_train_train = D_train[: d]
D_train_test = D_train[d: ]

In [27]:
dt_p = Decision_Tree_prepruning()
dt_p.train(D_train_train, A)
y_predict = dt_p.predict(D_test, A)
precision = np.sum(y_predict == D_test[:, -1]) / D_test.shape[0]
precision

0.8412698412698413

In [28]:
# 在训练集的基础上再以9:1的比例划分训练集中的训练集与测试集
np.random.shuffle(D_train)
d = int(D_train.shape[0] * 0.9) 
D_train_train = D_train[: d]
D_train_test = D_train[d: ]

In [29]:
dt_p = Decision_Tree_prepruning()
dt_p.train(D_train_train, A)
y_predict = dt_p.predict(D_test, A)
precision = np.sum(y_predict == D_test[:, -1]) / D_test.shape[0]
precision

0.6666666666666666